In [1]:
import torch
import pickle
from transformers import AutoTokenizer, AutoModel

In [2]:
# Configuration
MODEL_NAME = 'castorini/repllama-v1-7b-lora-passage' # replace with the actual RapLlaMA model ID
OUTPUT_PATH = "wsj_rapllama_embeddings.pkl"
MAX_LENGTH = 512  # model's max token length

In [4]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
model = get_model('castorini/repllama-v1-7b-lora-doc').to(device)
model.eval()


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

NameError: name 'get_model' is not defined

In [ ]:
def embed_text(text: str) -> torch.Tensor:
    """
    Generate a mean-pooled embedding for a single text string.
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

    mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    masked_states = hidden_states * mask
    summed = masked_states.sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    embeddings = summed / counts

    return embeddings.squeeze(0).cpu()


def embed_text_list(text_list: list) -> list:
    """
    Embed a list of text strings. Returns a list of numpy embedding arrays.
    """
    embeddings = []
    for idx, text in enumerate(text_list):
        emb = embed_text(text)
        embeddings.append(emb.numpy())
        print(f"Embedded doc {idx} -> shape: {emb.shape}")
    return embeddings


In [ ]:
wsj_texts = ['hello world!']
embeddings = embed_text_list(wsj_texts)